<a href="https://colab.research.google.com/github/MIHITA-KOSURU/Project/blob/main/StackOverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import csv

**Load the Dataset**

In [ ]:
survey = pd.read_csv('/content/drive/MyDrive/ENG to TEL/survey_results_public.csv')

In [ ]:
survey.head()

,ResponseId,Q120,MainBranch,Age,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,...,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,ProfessionalTech,Industry,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I agree,None of these,18-24 years old,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I agree,I am a developer by profession,25-34 years old,"Employed, full-time",Remote,Hobby;Contribute to open-source projects;Boots...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;Friend or fam...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,Never,15-30 minutes a day,15-30 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,285000.0
2,3,I agree,I am a developer by profession,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby;Professional development or self-paced l...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Formal documentation provided by the owner of ...,...,6-10 times a week,6-10 times a week,3-5 times a week,30-60 minutes a day,30-60 minutes a day,DevOps function;Microservices;Automated testin...,"Information Services, IT, Software Development...",Appropriate in length,Easy,250000.0
3,4,I agree,I am a developer by profession,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Colleague;Friend or family member;Other online...,Formal documentation provided by the owner of ...,...,1-2 times a week,10+ times a week,1-2 times a week,15-30 minutes a day,30-60 minutes a day,Automated testing;Continuous integration (CI) ...,NaN,Appropriate in length,Easy,156000.0
4,5,I agree,I am a developer by profession,25-34 years old,"Employed, full-time;Independent contractor, fr...",Remote,Hobby;Contribute to open-source projects;Profe...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Online Courses or Certi...,Formal documentation provided by the owner of ...,...,1-2 times a week,1-2 times a week,3-5 times a week,60-120 minutes a day,30-60 minutes a day,Microservices;Automated testing;Observability ...,Other,Appropriate in length,Neither easy nor difficult,23456.0


In [ ]:

print(survey.shape)

(89184, 84)


In [ ]:

survey['Employment'] = survey['Employment'].fillna('Selfemployed')


# Forward fill NaN values in the entire DataFrame
survey_filled = survey.fillna(method='ffill')

# Show the DataFrame after filling NaN values
print(survey_filled.head())


   ResponseId     Q120                      MainBranch              Age  \
0           1  I agree                   None of these  18-24 years old   
1           2  I agree  I am a developer by profession  25-34 years old   
2           3  I agree  I am a developer by profession  45-54 years old   
3           4  I agree  I am a developer by profession  25-34 years old   
4           5  I agree  I am a developer by profession  25-34 years old   

                                          Employment  \
0                                       Selfemployed   
1                                Employed, full-time   
2                                Employed, full-time   
3                                Employed, full-time   
4  Employed, full-time;Independent contractor, fr...   

                             RemoteWork  \
0                                   NaN   
1                                Remote   
2  Hybrid (some remote, some in-person)   
3  Hybrid (some remote, some in-person) 

In [ ]:
language_counter = Counter()
languages = survey['LanguageHaveWorkedWith']
# print(len(languages))
# print(languages.shape[0])

null_count = survey['LanguageHaveWorkedWith'].isnull().sum()
# print(null_count)

total = languages.shape[0] - null_count


for eachResponse in languages:
    try:
        split_ans = eachResponse.split(';')
    except AttributeError:
        split_ans = [str(eachResponse)]

    language_counter.update(split_ans)


for lan, val in language_counter.most_common(5):
    print(f"{lan}: {round(val/total * 100 , 2)}%")

JavaScript: 63.93%
HTML/CSS: 53.24%
Python: 49.53%
SQL: 48.91%
TypeScript: 39.06%


**Breaking popular languages by developer type**

In [ ]:
lan_dev_type = survey[['DevType', 'LanguageHaveWorkedWith']]

lan_dev_type = lan_dev_type.dropna(subset=['DevType']).reindex()

lan_dev_type['LanguageHaveWorkedWith'] = lan_dev_type['LanguageHaveWorkedWith'].fillna(
    'NA')

dev_type_info = {}


for dev_types, languages in zip(lan_dev_type['DevType'], lan_dev_type['LanguageHaveWorkedWith']):
    dev_type = dev_types.split(';')
    language = languages.split(';')
    for each_dev_type in dev_type:
        dev_type_info.setdefault(each_dev_type, {
            'total': 0,
            'language_counter': Counter()
        })
        dev_type_info[each_dev_type]['language_counter'].update(language)
        dev_type_info[each_dev_type]['total'] += 1


for dev_type, info in dev_type_info.items():
    print(dev_type)
    for language, value in info['language_counter'].most_common(5):
        language_pct = (value / info['total']) * 100
        language_pct = round(language_pct, 2)

        print(f"\t{language}: {language_pct}%")
    print()

Senior Executive (C-Suite, VP, etc.)
	JavaScript: 72.3%
	SQL: 59.83%
	HTML/CSS: 58.48%
	Python: 48.72%
	TypeScript: 45.05%

Developer, back-end
	SQL: 53.29%
	JavaScript: 50.11%
	Python: 44.29%
	Java: 37.62%
	HTML/CSS: 34.07%

Developer, front-end
	JavaScript: 91.62%
	HTML/CSS: 79.43%
	TypeScript: 70.42%
	SQL: 24.51%
	Python: 21.16%

Developer, full-stack
	JavaScript: 85.43%
	HTML/CSS: 71.84%
	SQL: 62.39%
	TypeScript: 59.85%
	Python: 39.57%

System administrator
	Python: 57.07%
	Bash/Shell (all shells): 54.1%
	SQL: 52.49%
	JavaScript: 52.09%
	HTML/CSS: 51.14%

Developer, desktop or enterprise applications
	SQL: 48.98%
	C#: 45.9%
	JavaScript: 41.39%
	Python: 37.12%
	C++: 36.35%

Developer, QA or test
	Python: 59.39%
	JavaScript: 53.41%
	Java: 40.61%
	SQL: 38.74%
	HTML/CSS: 37.88%

Designer
	HTML/CSS: 62.99%
	JavaScript: 61.57%
	Python: 30.25%
	SQL: 29.89%
	PHP: 23.49%

Data scientist or machine learning specialist
	Python: 93.64%
	SQL: 55.42%
	Bash/Shell (all shells): 35.64%
	JavaScript:

In [ ]:
import csv
from collections import defaultdict, Counter

with open('/content/drive/MyDrive/ENG to TEL/survey_results_public.csv', 'r', encoding='latin') as f:
    csv_reader = csv.DictReader(f)

    dev_type_info = {}

    for line in csv_reader:
        dev_types = line['DevType'].split(';')

        for dev_type in dev_types:
            dev_type_info.setdefault(dev_type, {
                'total': 0,
                'language_counter': Counter()
            })

            languages = line['LanguageHaveWorkedWith'].split(';')
            dev_type_info[dev_type]['language_counter'].update(languages)
            dev_type_info[dev_type]['total'] += 1


for dev_type, info in dev_type_info.items():
    print(dev_type)

    for language, value in info['language_counter'].most_common(5):
        language_pct = (value / info['total']) * 100
        language_pct = round(language_pct, 2)

        print(f'\t{language}: {language_pct}%')

NA
	Python: 59.04%
	JavaScript: 54.57%
	HTML/CSS: 52.63%
	C++: 35.17%
	Java: 35.13%
Senior Executive (C-Suite, VP, etc.)
	JavaScript: 72.3%
	SQL: 59.83%
	HTML/CSS: 58.48%
	Python: 48.72%
	TypeScript: 45.05%
Developer, back-end
	SQL: 53.29%
	JavaScript: 50.11%
	Python: 44.29%
	Java: 37.62%
	HTML/CSS: 34.07%
Developer, front-end
	JavaScript: 91.62%
	HTML/CSS: 79.43%
	TypeScript: 70.42%
	SQL: 24.51%
	Python: 21.16%
Developer, full-stack
	JavaScript: 85.43%
	HTML/CSS: 71.84%
	SQL: 62.39%
	TypeScript: 59.85%
	Python: 39.57%
System administrator
	Python: 57.07%
	Bash/Shell (all shells): 54.1%
	SQL: 52.49%
	JavaScript: 52.09%
	HTML/CSS: 51.14%
Developer, desktop or enterprise applications
	SQL: 48.98%
	C#: 45.9%
	JavaScript: 41.39%
	Python: 37.12%
	C++: 36.35%
Developer, QA or test
	Python: 59.39%
	JavaScript: 53.41%
	Java: 40.61%
	SQL: 38.74%
	HTML/CSS: 37.88%
Designer
	HTML/CSS: 62.99%
	JavaScript: 61.57%
	Python: 30.25%
	SQL: 29.89%
	PHP: 23.49%
Data scientist or machine learning specialis